# ラボ 1: ReAct エージェントをゼロから構築する

## ReAct パターン

このセクションでは、ReAct (推論と動作) パターンを使用して AI エージェントを構築します。この概念に馴染みがなくても心配はいりません。ステップごとに説明します。

ReAct パターンは、人間の認知パターンを反映した AI の問題解決プロセスを構造化するフレームワークです。

1. 現在の状況について **推論** する
2. 実行するアクションを **決定** する
3. そのアクションの結果を **観察** する
4. タスクが完了するまで **繰り返す**

この概念を説明するために、経験豊富なソフトウェア エンジニアが複雑なシステムのデバッグにどのように取り組むかを考えてみましょう。

1. **推論**: エラー ログとシステム状態を分析する (例: 「データベース接続がタイムアウトしています」)
2. **行動**: 診断アクションを実装する (例: 「データベース接続テストを実行する」)
3. **観察**: 診断の結果を調べる (例: 「テストでレイテンシが高いことが示されています」)
4. **繰り返す**: 問題が解決するまで、このプロセスを継続し、次にネットワーク構成を確認するなどします

当社の AI エージェントは、同様の方法論を使用して問題に取り組みます。このエージェントを開発する際は、AI モデル (推論して決定する「頭脳」) と Python コード (環境とやり取りしてプロセス フローを管理する「本体」) の間の分担に注意してください。

このノートブックは、次の [Simon Willison のノートブック](https://til.simonwillison.net/llms/python-react-pattern) に基づいています。

## 環境の設定

まず、必要なライブラリをインポートして環境を構成しましょう。

### Bedrock クライアントの初期化

Amazon Bedrock 経由で Claude モデルと通信するには、クライアント接続を確立する必要があります。このクライアントは、コードが AI モデルにリクエストを送信し、応答を受信できるようにする API ゲートウェイと考えてください。

ここでは、Python 用の Amazon Web Services (AWS) SDK である `boto3` ライブラリを使用します。AWS に詳しくない方のために説明すると、`boto3` は、Bedrock を含むさまざまな AWS サービスと Python のやり取りを容易にする包括的なツールキットと考えることができます。

AWS 認証情報を使用して `boto3` を構成するための包括的な手順については、[AWS ドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html) を参照してください。

本番環境では、安全な AWS 認証情報管理を実装します。このセクションでは、資格情報が環境で事前に構成されていると想定します。

In [ ]:
from dotenv import load_dotenv
import os
import sys
import boto3
import re
from botocore.config import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client in your aws region.
# If you do not have the AWS CLI profile setup, you can authenticate with aws access key, secret and session token.
# For more details check https://docs.aws.amazon.com/cli/v1/userguide/cli-authentication-short-term.html
bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    config=bedrock_config,
)

まず、いくつかの推論パラメータを定義し、`boto3` 経由で Amazon Bedrock への接続をテストします。

In [ ]:
# Set inference parameters
temperature = 0.0
top_k = 200
inference_config = {"temperature": temperature}

additional_model_fields = {"top_k": top_k}
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
system_prompts = [{"text": "You are a helpful agent."}]
message_1 = {"role": "user", "content": [{"text": "Hello world"}]}

# Instantiate messages list
messages = []
messages.append(message_1)

# Send the message.
response = bedrock_rt.converse(
    modelId=model_id,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields,
)

pp.pprint(response)
print("\n\n")
pp.pprint(response["output"]["message"]["content"][0]["text"])

## エージェント クラスの設計

Bedrock クライアントをセットアップしたら、エージェント クラスを作成します。このクラスは AI エージェントの中核として機能し、Claude モデルと対話して会話状態を維持するためのロジックをカプセル化します。

エージェントが実装する ReAct パターンは、3 つの主要なステップで構成されます。

1. **推論 (思考)**: エージェントは現在の状況を評価して計画を策定します。たとえば、「2 種類の犬種の合計体重を計算するには、それぞれの体重を調べて合計する必要があります。」

2. **行動 (アクション)**: エージェントは推論に基づいて適切なアクションを選択します。たとえば、「ボーダー コリーの平均体重を照会します。」

3. **観察 (観察)**: エージェントはアクションからのフィードバックを処理します。この場合、「ボーダー コリーの平均体重は 30 ～ 55 ポンドです。」などです。

このパターンにより、エージェントは複雑なタスクを管理可能なステップに分解し、新しい情報に基づいて戦略を適応させることができます。

エージェント クラスは、会話履歴 (`self.messages`) を維持し、Claude モデルと対話するメソッド (`__call__` および `execute`) を提供することで、このパターンを実装します。

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.system = [{"text": self.system}]
        self.bedrock_client = boto3.client(service_name="bedrock-runtime")

    def __call__(self, message):
        self.messages.append({"role": "user", "content": [{"text": message}]})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": [{"text": result}]})
        return result

    def execute(self):
        inference_config = {
            "temperature": 0.0,
            "stopSequences": [
                "<PAUSE>"
            ],  # we will explore later why this is important!
        }

        additional_model_fields = {"top_k": 200}

        response = self.bedrock_client.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=self.messages,
            system=self.system,
            inferenceConfig=inference_config,
            additionalModelRequestFields=additional_model_fields,
        )
        return response["output"]["message"]["content"][0]["text"]

## プロンプトの作成

プロンプトは AI モデルに対する一連の指示として機能し、その動作と利用可能なアクションを定義する上で重要です。

実装では、モデルに次のことを指示しています。

- ReAct パターン (思考、アクション、観察のサイクル) に準拠する
- 各ステップで特定の形式を使用する (思考の前に「思考:」を付けるなど)
- 提供されたアクションに制限する (この場合は、計算機と犬の体重検索機能)

また、予想される応答形式を示すサンプル インタラクションも含まれています。これは、複雑なフォームに記入するように依頼する前に、完成したテンプレートを提供するのと似ています。

プロンプト、エージェント クラス、および推論パラメータからわかるように、モデルに `<PAUSE>` トークンを予測した後で生成を停止するように要求しています。ただし、安全策を講じる方が常に良いため、`stopSequences` に `<PAUSE>` を追加して、その後のトークン生成の終了を確実にします。

In [ ]:
prompt = """
You run in a loop of Thought, Action, <PAUSE>, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

If available, always call a tool to inform your decisions, never use your parametric knowledge when a tool can be called. 

When you have decided that you need to call a tool, output <PAUSE> and stop thereafter! 

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
<PAUSE>
----- execution stops here -----
You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

> 追記　プロンプトの翻訳

```
思考、アクション、<PAUSE>、観察のループを実行します。
ループの最後に回答を出力します
思考を使用して、尋ねられた質問に対する考えを説明します。
アクションを使用して、使用可能なアクションの 1 つを実行し、PAUSE を返します。
観察は、これらのアクションを実行した結果になります。

使用可能なアクションは次のとおりです:

計算:
例: 計算: 4 * 7 / 3
計算を実行して数値を返します。Python を使用するため、必要に応じて浮動小数点構文を使用してください

平均犬の体重:
例: 平均犬の体重: コリー
犬種が指定された場合の平均体重を返します

使用可能な場合は、常にツールを呼び出して決定を通知し、ツールを呼び出せる場合はパラメトリックの知識を使用しないでください。

ツールを呼び出す必要があると判断した場合は、<PAUSE> を出力してその後停止します。

セッションの例:

質問: ブルドッグの体重はどれくらいですか?
考え: average_dog_weight を使用して犬の体重を調べる必要があります
アクション: average_dog_weight: Bulldog
<PAUSE>
----- ここで実行が停止します -----
次のように再度呼び出されます:

観察: ブルドッグの体重は 51 ポンドです

次に出力します:

回答: ブルドッグの体重は 51 ポンドです
```

## ヘルパー関数の実装

エージェントに実用的な機能を持たせるために、いくつかのヘルパー関数を定義します。これらの関数は、エージェントが実行できる「アクション」として機能します。この例では、次のものを提供しています。

1. 基本的な計算関数
2. 犬の平均体重を取得する関数

より高度なアプリケーションでは、これらの関数は、Web スクレイピングからデータベース クエリ、API 呼び出しまで、さまざまな操作をカバーできます。これらは、エージェントと外部データ ソースおよびシステムとの多目的インターフェイスであり、幅広い可能性を提供します。

In [ ]:
def calculate(what):
    return eval(what, {"__builtins__": {}}, {})


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

## エージェントのテスト

エージェントとそのアクション セットを定義したら、トイプードルの体重に関する簡単なクエリを使用して初期テストを実行します。

このテストでは、エージェントの情報処理フローを明らかにします。

1. 必要な手順について推論します (体重を調べる必要があることを識別します)
2. アクションを実行します (`average_dog_weight` 関数を呼び出します)
3. 観察を処理します (トイプードルの返された体重)
4. この情報を統合して一貫した応答を作成します

In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

In [ ]:
result = average_dog_weight("Toy Poodle")
result

In [ ]:
next_prompt = "Observation: {}".format(result)

In [ ]:
abot(next_prompt)

In [ ]:
abot.messages

In [ ]:
abot = Agent(prompt)
abot.messages

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(eval("37 + 20"), {"__builtins__": {}}, {})
print(next_prompt)

In [ ]:
abot(next_prompt)

### stopSequences について

上記のエージェント クラスから `stopSequence` パラメータを削除してみてください。

次に進む前に、以下の質問について考えてみましょう:

- エージェントのパフォーマンスは今どうですか?
- `stopSequences` はいつ使用すべきですか? いつアプリケーションの負担になることがありますか?
- プロンプトは [Anthropic のプロンプト標準](https://docs.anthropic.com/en/docs/prompt-engineering) に準拠していますか?

ノートブックの最後に、エージェントが `stopSequences` を使用せずに指示に従うように演習を完了してみてください。

## 推論ループの実装

エージェントの自律性を高めるために、エージェントが答えを求めて複数回推論、行動、観察できるようにする反復ループを実装します。このループは、エージェントが結論に達するか、事前に定義された反復の最大回数に達するまで続きます。

このアプローチは、人間の専門家が複雑な問題に取り組む方法を反映しており、情報を収集し、解決策に到達するまで複数の手順を実行します。このループにより、エージェントは複数の手順またはデータ ポイントを必要とするより複雑なクエリを処理できるようになります。

In [ ]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

## 最終評価

最後に、複数のステップの推論とアクションを必要とするより複雑なクエリを使用して、完全に実装されたエージェントをテストします。2 つの異なる犬種の合計重量を計算するタスクをエージェントに課します。

この包括的なテストでは、エージェントの次の機能を紹介します。

1. 複雑なクエリを管理可能なサブタスクに分解する
2. 複数の犬種の情報を取得する
3. 収集したデータを使用して計算を実行する
4. このすべての情報を統合して、一貫した最終応答を作成する

この実用的な例に取り組むことで、複数のステップの問題を解決できる AI エージェントの構築に関する貴重な洞察が得られます。さらに、Amazon Bedrock や Anthropic の Claude などのモデルプロバイダーを効果的に活用する方法を直接確認できます。この知識により、将来のプロジェクトでより柔軟で多様な AI アプリケーションを開発できるようになります。

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

# 演習 - エージェントを書き直して、人間中心のスタイルのプロンプトを使用します。